In [121]:
import pandas as pd
import numpy as np
import os
import collections
from sklearn.decomposition import PCA
import time

In [8]:
datapath = os.path.join(os.getcwd(),"tweet_data")

In [156]:
def get_hashtag_usage_frequency(joined_data):
    hashtags = []
    a = joined_data["lc_hashtags"].tolist()
    for h in range(len(a)):
        if(a[h]):
            for i in a[h]:
                hashtags.append(i)
    hashtags = [x.lower() for x in hashtags]
    hashtag_frequency = collections.Counter(hashtags)
    hashtag_frequency = hashtag_frequency.most_common()
    return(hashtag_frequency)

def get_match(a,control,w):

    if(w % 50 == 0):
        print("We have completed ", w, " matchings")
    cosin_sims = [np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)) for b in control]
    control_index = cosin_sims.index(max(cosin_sims))
    
    return(control_index)

In [10]:
treated = pd.read_csv(os.path.join(datapath,"treated_df.csv"))
control = pd.read_csv(os.path.join(datapath,"control_df.csv"))

In [28]:
treated["lc_hashtags"] = [np.matrix(w).A[0].tolist() for w in treated["lc_hashtags"]]

In [33]:
used_hashs = get_hashtag_usage_frequency(treated)

In [147]:
low = np.percentile([j[1] for j in used_hashs],[95.5,96])[0]
high = np.percentile([j[1] for j in used_hashs],[95.5,96])[1]
relevant_hashs = [h[0] for h in used_hashs if h[1] >= low and h[1] <= high]

In [149]:
kept_treatment_observations = [h for h in range(len(treated)) if 
 any(j in relevant_hashs for j in treated.iloc[h].lc_hashtags)]

In [150]:
treated = treated.iloc[kept_treatment_observations]

In [77]:
treated.word_vectors = [np.matrix(w).A[0].tolist() for w in treated.word_vectors]

In [78]:
control.word_vectors = [np.matrix(w).A[0].tolist() for w in control.word_vectors]

In [152]:
a = pd.DataFrame(treated.reset_index().word_vectors.tolist())
b = pd.DataFrame(control.reset_index().word_vectors.tolist())
pca = PCA(n_components=10)
x = pd.concat([a, b], ignore_index=True)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)
a = principalDf.iloc[0:len(treated)].values.tolist()
b = principalDf.iloc[len(treated):].values.tolist()

In [153]:
treated["pca_reduced_vectors"] = a
control["pca_reduced_vectors"] = b

In [157]:
start = time.time()
control_vectors = [get_match(treated.iloc[w]["pca_reduced_vectors"],
                             control["pca_reduced_vectors"],w) for w in range(len(treated))]
end = time.time()
print("This took ", end - start, " seconds")

We have completed  0  matchings
We have completed  50  matchings
We have completed  100  matchings
We have completed  150  matchings
We have completed  200  matchings
We have completed  250  matchings
We have completed  300  matchings
We have completed  350  matchings
We have completed  400  matchings
We have completed  450  matchings
We have completed  500  matchings
We have completed  550  matchings
We have completed  600  matchings
We have completed  650  matchings
We have completed  700  matchings
We have completed  750  matchings
We have completed  800  matchings
We have completed  850  matchings
We have completed  900  matchings
We have completed  950  matchings
We have completed  1000  matchings
We have completed  1050  matchings
We have completed  1100  matchings
We have completed  1150  matchings
We have completed  1200  matchings
We have completed  1250  matchings
We have completed  1300  matchings
We have completed  1350  matchings
We have completed  1400  matchings
We have 

In [167]:
treated["control_match"] = control_vectors
c = pd.DataFrame(control_vectors)
c.to_csv(os.path.join(datapath,"matched_control_vectors_df.csv"))
matched_controls = control.iloc[control_vectors]
matched_controls["lc_hashtags"] = [np.matrix(w).A[0].tolist() for
                                w in matched_controls["lc_hashtags"]]

In [196]:
control_matched_unique_hashtags = get_hashtag_usage_frequency(matched_controls)
treated_unique_hashtags = get_hashtag_usage_frequency(treated)
treated_unique_hashtags = pd.DataFrame(treated_unique_hashtags,columns = ["hashtag","count"])
control_matched_unique_hashtags = pd.DataFrame(control_matched_unique_hashtags,
                                               columns = ["hashtag","count"])

In [192]:
print(len(control_matched_unique_hashtags))
len(treated_unique_hashtags)

539


2263

In [201]:
treated_unique_hashtags.hashtag.tolist()
control_matched_unique_hashtags.hashtag.tolist()
len(list(set(treated_unique_hashtags.hashtag.tolist()) -
     set(control_matched_unique_hashtags.hashtag.tolist())))

2030

In [226]:
treated["user_id"] = [ast.literal_eval(u).get("id") for u in treated.user]
matched_controls["user_id"] = [ast.literal_eval(u).get("id") for u in matched_controls.user]

<ipython-input-226-5b7711ed8bb3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_controls["user_id"] = [ast.literal_eval(u).get("id") for u in matched_controls.user]


In [228]:
treated.to_csv(os.path.join(datapath,"tweets","DAPL","final_treated_df.csv"))
matched_controls.to_csv(os.path.join(datapath,"tweets","DAPL","final_control_df.csv"))

In [219]:
os.path.join(datapath,"tweets","DAPL","final_control_df.csv")

'C:\\Users\\Poet\\usc_coursework\\spring-2022\\fairness-in-ai\\project\\tweet_data\\tweets\\DAPL\\final_control_df.csv'

In [218]:
import ast
test_string = treated.user.iloc[0]
#json.loads()
ast.literal_eval(test_string).get("id")
#.get("id")

17184930